analyze He's data to see how much ROH there is.
Turns out there isn't much.
In the two minimally contaminated male samples, there are only about 25cM of ROH genome-wide, I don't think that's enough for reliable inference.
I think the Carribean dataset is a better alternative.

In [1]:
iid="GFW002.003"

In [1]:
# convert mpileup to hdf5
from toHDF5 import mpileup2hdf5

basepath=f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}"
for ch in range(1, 23):
    print(f'processing chr{ch}')
    mpileup2hdf5(f'{basepath}/mpileup/{iid}.chr{ch}.mpileup', 
        f'/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr{ch}.hdf5',
        iid=iid, outPath=f'{basepath}/hdf5/')


processing chr1
number of major reads at flanking sites: 1856166.0
number of minor reads at flanking sites: 10789.0
number of major reads at focal sites: 113261.0
number of minor reads at focal sites: 8654.0
err rate at flanking sites: 0.005778928790463616
err rate at focal sites: 0.07098388221301727
saving sample as GFW002.003 in /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hdf5/GFW002.003.chr1.hdf5
processing chr2
number of major reads at flanking sites: 1763038.0
number of minor reads at flanking sites: 8313.0
number of major reads at focal sites: 124623.0
number of minor reads at focal sites: 9271.0
err rate at flanking sites: 0.004693028089859096
err rate at focal sites: 0.06924134016460783
saving sample as GFW002.003 in /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hdf5/GFW002.003.chr2.hdf5
processing chr3
number of major reads at flanking sites: 1395233.0
number of minor reads at flanking sites: 5010.0
number of major reads at focal sites: 123298.0
number of minor r

In [2]:
import sys
sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  # hack to get local package first in path [FROM HARALD - DELETE!!!]
from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind  # Need this import
from hapsburg.PackagesSupport.hapsburg_run import hapsb_chunk_negloglik

In [7]:
hapsb_ind(iid, chs=range(1,23), 
    path_targets_prefix = f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hdf5",
    h5_path1000g = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    meta_path_ref = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    folder_out=f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/", prefix_out="",
    e_model="readcount", p_model="SardHDF5", post_model="Standard",
    processes=4, delete=False, output=True, save=True, save_fp=False, 
    n_ref=2504, diploid_ref=True, exclude_pops=[], readcounts=True, random_allele=False,
    roh_in=1, roh_out=20, roh_jump=300, e_rate=0.01, e_rate_ref=1e-3, 
    cutoff_post = 0.999, max_gap=0, roh_min_l = 0.04, logfile=True, combine=True, 
    file_result="_roh_full.csv")

Doing Individual GFW002.003...
Running 22 total jobs; 4 in parallel.
Starting Pool of multiple workers...
Set Output Log path: /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hapRoh/GFW002.003/chr1/hmm_run_log.txtSet Output Log path: /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hapRoh/GFW002.003/chr5/hmm_run_log.txtSet Output Log path: /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hapRoh/GFW002.003/chr3/hmm_run_log.txtSet Output Log path: /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hapRoh/GFW002.003/chr7/hmm_run_log.txt





KeyboardInterrupt: 

In [9]:
hapsb_chunk_negloglik(iid, 3, 1000, -1000, 
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hdf5/{iid}.chr3.hdf5",
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/",
    0.0, roh_in=1, roh_out=20, roh_jump=300, e_rate=0.01, e_rate_ref=0.0,
    save=False, save_fp=False, n_ref=2504, diploid_ref=True, 
    exclude_pops=[], e_model="readcount_contam", p_model="SardHDF5", 
    readcounts=True, random_allele=False,
    post_model="Standard", prefix_out="", logfile=False)

Using Rescaled HMM.
Loaded Pre Processing Model: SardHDF5
Loading Individual: GFW002.003

Loaded 58677 variants
Loaded 1 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/He/contamX/GFW002.003/hdf5/GFW002.003.chr3.hdf5

Loaded 77652 variants
Loaded 2504 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr3.hdf5
subsetting reference genotype... starting from 1000th marker until -1000th marker.

Intersection on Positions: 58677
Nr of Matching Refs: 58677 / 58677
Ref/Alt Allele Matching: 58677 / 58677
Flipped Ref/Alt Alleles for 0 SNPs
Together: 58677 / 58677
2504 / 2504 Individuals included in Reference
Extracting up to 2504 Individuals
including all individuals from the reference panel as the contamination population
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (2, 58677)
Extraction of 2 Haplotypes complete
Flipping Ref/Alt Allele in target for 0 SNPs...
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (5008, 58

26043.443786579654

In [ ]:
# compare results to make sure my code for hapsb_chunk_negloglik is correct